Given that all three classifiers are showing similar and relatively poor performance, it's indeed a good idea to consider revisiting the overall approach. Here are some potential steps and strategies to improve the performance:

### Strategies for Improvement

1. **Data Quality and Preprocessing**:
   - **Check for Class Imbalance**: Ensure that the classes are balanced. If not, consider techniques like SMOTE or other resampling methods.
   - **Feature Engineering**: Explore additional features or different feature extraction techniques that might capture more relevant information from the audio signals.
   - **Noise Reduction**: Apply more advanced noise reduction techniques to improve the signal quality.

2. **Model Architecture and Hyperparameters**:
   - **CNN Architecture**: Experiment with more complex CNN architectures or additional layers such as LSTM for capturing temporal dependencies.
   - **Hyperparameter Tuning**: Perform hyperparameter optimization for all models using techniques like Grid Search or Random Search.

3. **Ensemble Methods**:
   - **Stacking**: Use more sophisticated stacking methods with a meta-learner that might capture the strengths of each classifier better.
   - **Bagging and Boosting**: Explore other ensemble methods like XGBoost or AdaBoost.

4. **Additional Data**:
   - **Augmentation**: Increase the amount of training data through data augmentation techniques.
   - **External Data**: Incorporate additional data sources if available to provide more context and variability.

### Detailed Steps for Improvement

#### 1. Data Quality and Preprocessing

##### Check for Class Imbalance

```python
from collections import Counter

# Check class distribution in the training set
y_train_labels = label_encoder.inverse_transform(y_train)
class_distribution = Counter(y_train_labels)
print("Class Distribution in Training Set:", class_distribution)

# Visualize class distribution
import matplotlib.pyplot as plt

plt.bar(class_distribution.keys(), class_distribution.values())
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.title('Class Distribution in Training Set')
plt.xticks(rotation=90)
plt.show()
```

##### Feature Engineering and Noise Reduction

Explore additional feature extraction techniques or advanced noise reduction methods.

```python
import librosa

# Example: Spectrogram features
def extract_spectrogram_features(segment, sr, max_length):
    spectrogram = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=128, fmax=8000)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
    spectrogram_flat = spectrogram_db.flatten()
    if len(spectrogram_flat) < max_length:
        padded_spectrogram = np.pad(spectrogram_flat, (0, max_length - len(spectrogram_flat)), mode='constant')
        return padded_spectrogram
    else:
        return spectrogram_flat[:max_length]

# Apply new feature extraction
feature_data_spectrogram = []
max_length_spectrogram = 0
for file_name, command, segment, sr in all_segments:
    features = extract_spectrogram_features(segment, sr, max_length_spectrogram)
    feature_data_spectrogram.append((file_name, command, features))
    if len(features) > max_length_spectrogram:
        max_length_spectrogram = len(features)

# Prepare data for training with new features
X_spectrogram = np.array([features for _, _, features in feature_data_spectrogram])
y_spectrogram = np.array([command for _, command, _ in feature_data_spectrogram])
```

#### 2. Model Architecture and Hyperparameters

##### Experiment with More Complex CNN Architectures

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define a more complex CNN model
def create_complex_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the complex CNN model
input_shape = (height, width, 1)
num_classes = len(np.unique(y_train_cnn.argmax(axis=1)))
complex_cnn_model = create_complex_cnn_model(input_shape, num_classes)

# Train the complex CNN model
complex_cnn_model.fit(X_train_cnn, y_train_cnn, epochs=20, batch_size=32, validation_data=(X_val_cnn, y_val_cnn))
```

##### Hyperparameter Tuning

```python
from sklearn.model_selection import GridSearchCV

# Example: Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search_rf.fit(X_train, y_train)

print("Best parameters for Random Forest:", grid_search_rf.best_params_)
print("Best score for Random Forest:", grid_search_rf.best_score_)
```

#### 3. Ensemble Methods

Consider more sophisticated ensembling techniques.

```python
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Define base classifiers with the best parameters from tuning
estimators = [
    ('rf', RandomForestClassifier(**grid_search_rf.best_params_)),
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42))
]

# Define stacking classifier
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_clf.fit(X_train, y_train)

# Make predictions
y_pred_stacking = stacking_clf.predict(X_val)

# Evaluate the stacking classifier
print("Stacking Classifier Report")
print(classification_report(y_val, y_pred_stacking, zero_division=0))
```

### Summary

1. **Data Quality and Preprocessing**: Address class imbalance, explore new features, and apply advanced noise reduction.
2. **Model Architecture and Hyperparameters**: Experiment with more complex CNN architectures and perform hyperparameter tuning.
3. **Ensemble Methods**: Use more sophisticated ensemble techniques like stacking.

By implementing these strategies, you should be able to significantly improve the performance of your classifiers. Let me know if you need further assistance with any of these steps!